In [1]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, MarianMTModel, MarianTokenizer
import random
import pandas as pd

def set_seed(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)
def generate_variante(sentence):
    model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_paraphraser')
    tokenizer = T5Tokenizer.from_pretrained('t5-base')

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    text =  "paraphrase: " + sentence + " </s>"


    max_len = 256

    encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
    input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)


    # set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
    beam_outputs = model.generate(
        input_ids=input_ids, attention_mask=attention_masks,
        do_sample=True,
         max_length=256,
        top_k=120,
        top_p=0.98,
        early_stopping=True,
        num_return_sequences=10
      )



    final_outputs =[]
    for beam_output in beam_outputs:
        sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
        if sent.lower() != sentence.lower() and sent not in final_outputs:
                  final_outputs.append(sent)
  
    # Charger le modèle de traduction anglais-français
    translator = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
    tokenizer_fr = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

    # Traduire chaque paraphrase en français
    paraphrases_fr = []
    for paraphrase in final_outputs:
        # Ajouter un préfixe "translate English to French: " pour indiquer au modèle la direction de traduction
        text = paraphrase
        # Tokenization pour le modèle de traduction
        inputs = tokenizer_fr.encode(text, return_tensors="pt", padding=True)
        # Utilisation du modèle pour obtenir la traduction
        translated = translator.generate(inputs.to(device))
        # Décodage de la traduction en texte lisible
        paraphrase_fr = tokenizer_fr.decode(translated[0], skip_special_tokens=True)
        paraphrases_fr.append(paraphrase_fr)
    
    new_question = random.choice(paraphrases_fr)
    return new_question


In [2]:
def apply_function_to_column(data_frame, column_name, function_to_apply):
    new_data_frame = data_frame.copy()
    new_data_frame[column_name] = new_data_frame[column_name].apply(function_to_apply)
    return new_data_frame

In [3]:
# Lire le fichier CSV
csv_file_path = r"C:/Users/USER/Downloads/DATAUpdate1-1 (1).csv"
df = pd.read_csv(csv_file_path)

# Mélanger les lignes du DataFrame de manière aléatoire
df_shuffled = df.sample(frac=1, random_state=42)

# Sélectionner 30 lignes aléatoires
num_rows_to_select = 30
selected_rows = df_shuffled.sample(n=num_rows_to_select, random_state=42)

# Appliquer la fonction generate_variante à la colonne 'question' des 30 lignes sélectionnées
selected_rows_with_variants = apply_function_to_column(selected_rows, 'question', generate_variante)

# Afficher les lignes sélectionnées avec les paraphrases générées
print(selected_rows_with_variants)

C:\Users\USER\anaconda3\lib\site-packages\transformers\models\t5\tokenization_t5.py:199: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request availabl

                                              question  \
176  111. Parmi les mots suivants, qui n'a jamais e...   
233  47. L-humoriste français Coluche a déformé le ...   
51   74. Les deux phrases suivantes sont correctes....   
77             • La femme révèle l'intrus. Trouvez-le.   
215  Peut-on infiter des jambes, et avant qu'il par...   
7    21. Avec quel mot jumele le verbe voie dans la...   
188  Autre: 22. Le nom carpaccio vient d'un nom pro...   
12     Que voulez-vous dire sur l'élu(e) de votre cur?   
143  Cherchez la langue d'origine des noms et vous ...   
183  Ces fromages fatigués leur nom de la localité ...   
66   (les noms de ces déterminants (adjectifs numér...   
21   Entrez la lettre des phrases qui ne sont pas à...   
173  104. Un point de vue étylogique, de quel mot s...   
49   Quelle est la vérité de chaque phrase qui a ét...   
144  Pouvez-vous être attesté par la première attes...   
121  • Parmi ces adjectifs, lequel est synonyme de ...   
17           4

In [5]:
print(type(selected_rows_with_variants))

<class 'pandas.core.frame.DataFrame'>


In [5]:
selected_rows_with_variants.head()

,question,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,Column11,Column12
176,"111. Parmi les mots suivants, qui n'a jamais e...",A. visa,B. agenda,C. crocus,D. motus,NaN,NaN,NaN,NaN,NaN,110. d. Motus est une latinisation plaisante d...,NaN
233,47. L-humoriste français Coluche a déformé le ...,"A. L’argent ne fait pas le bonheur, mais il y ...",B. L’argent ne fait pas le bonheur des pauvres.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47. b. À «L’argent ne fait pas le bonheur des ...,NaN
51,74. Les deux phrases suivantes sont correctes....,A. Cette randonnée n’était pas aussi difficile...,B. Cette randonnée n’était pas aussi difficile...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74. Vrai. On peut reprendre ou non Cette rando...,NaN
77,• La femme révèle l'intrus. Trouvez-le.,A. artisan,B. médian,C. paysan,D. roman,NaN,NaN,NaN,NaN,NaN,165. c. Tous les noms et adjectifs qui se term...,NaN
215,"Peut-on infiter des jambes, et avant qu'il par...",A. d’escampette,B. de riz,C. de perlimpinpin,NaN,NaN,NaN,NaN,NaN,NaN,"40. a. Le nom escampette, qu’on ne trouve plus...",NaN


In [6]:
# Afficher les paraphrases générées au format souhaité
for index, row in selected_rows_with_variants.iterrows():
    print(f"{index + 1} - Question: {row['question']}")
    print("Options de paraphrase:")
    for i in range(2, 11):  # Boucler à travers les colonnes de Column2 à Column10
        option = row[f'Column{i}']
        if isinstance(option, str):
            print(f"Option {i - 1}: {option}")
    correct_answer = row['Column11']
    print(f"Correct Answer: {correct_answer}\n")


177 - Question: 111. Parmi les mots suivants, qui n'a jamais existé en latin?
Options de paraphrase:
Option 1: A. visa
Option 2: B. agenda
Option 3: C. crocus
Option 4: D. motus
Correct Answer: 110. d. Motus est une latinisation plaisante du nom mot.

234 - Question: 47. L-humoriste français Coluche a déformé le proverbe L-humoriste ne fait pas visage.
Options de paraphrase:
Option 1: A. L’argent ne fait pas le bonheur, mais il y contribue.
Option 2: B. L’argent ne fait pas le bonheur des pauvres.
Correct Answer: 47. b. À «L’argent ne fait pas le bonheur des pauvres», Coluche ajoutait:

52 - Question: 74. Les deux phrases suivantes sont correctes. vrai ou faux?
Options de paraphrase:
Option 1: A. Cette randonnée n’était pas aussi difficile que je le pensais.
Option 2: B. Cette randonnée n’était pas aussi difficile que je pensais.
Correct Answer: 74. Vrai. On peut reprendre ou non Cette randonnée n’était pas aussi difficile par le pronom le.

78 - Question: • La femme révèle l'intrus. T